## 一、获取并处理数据得到时间序列

In [1]:
import numpy as np
import pandas as pd
import tushare as ts

ts.set_token('094f15d71394516b730602faa77b1c708007b8d05df300590b4445ed')
pro = ts.pro_api()
# 多元化银行
diverse_bank_list = ['000001.SZ','600000.SH','600015.SH','600016.SH','600036.SH','601166.SH',
                     '601288.SH','601328.SH','601398.SH','601818.SH',
                     '601939.SH','601988.SH','601998.SH']
# 区域性银行
regional_bank_list = ['002142.SZ','002807.SZ','002839.SZ','002936.SZ','002948.SZ','002958.SZ',
                      '002966.SZ','600908.SH','600919.SH','600926.SH','600928.SH','601009.SH',
                      '601077.SH','601128.SH','601169.SH','601229.SH','601577.SH','601838.SH',
                      '601860.SH','601997.SH','603323.SH']
# df_1 = pro.daily(ts_code='000001.SZ', start_date='20190101', end_date='20191231')
# df_1.head()



def get_stock(name, start_date, end_date, N_amount=1):
    '''
    日交易额单位：千元（日成交额是否需要改变单位？）
    
    
    
    Args:
        name: stock name
        start_date: trade start date
        end_date: trade end date
        N_amount: 成交额的单位 
    
    '''
    df = pro.daily(ts_code=name, start_date=start_date, end_date=end_date)
    df['yield_rate'] = df['change']/df['pre_close']
    df['abs_yield_rate'] = abs(np.log(df['close']/df['pre_close']))
    df['amihud'] = df['abs_yield_rate']/(df['amount']/N_amount)
    return df



def process_stock(name, df):
    df = df[['trade_date','yield_rate','amihud']]
    df.rename(columns={'yield_rate':'yield_rate_'+name, 'amihud':'amihud_'+name},inplace=True)
    return df
    
    
    
def create_time_series(stock_list, start_date, end_date, N_amount=1):
    '''
    获取 stock_list 中每只 stock 的 yield rate 以及 amihud 指标并得到时间序列
    
    Args:
    
    '''
    # 先获取第一个
    df = get_stock(stock_list[0], start_date, end_date, N_amount)
    df = process_stock(stock_list[0], df)
    # 将第一个和后面的合并
    for i in range(len(stock_list)-1):
        df_temp = get_stock(stock_list[i+1], start_date, end_date, N_amount)
        df_temp = process_stock(stock_list[i+1], df_temp)
        df = pd.merge(df,df_temp)
    return df

def main():
    # 成交额的单位为百万
    df = create_time_series(diverse_bank_list,'20190101','20191231', 1000)
    return df
    

if __name__ == '__main__':
    df = main()
    ## 改变顺序
    row_list = ['trade_date']
    for name in diverse_bank_list:
        row_list.append('yield_rate_'+name)
    for name in diverse_bank_list:
        row_list.append('amihud_'+name)
    df = df[row_list]
    df.to_csv('stock1.csv', index=False)

ConnectionError: HTTPConnectionPool(host='api.waditu.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000241E5ADA488>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [7]:
df.index = pd.to_datetime(df.trade_date)
df

,trade_date,yield_rate_000001.SZ,yield_rate_600000.SH,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601166.SH,yield_rate_601288.SH,yield_rate_601328.SH,yield_rate_601398.SH,...,amihud_600016.SH,amihud_600036.SH,amihud_601166.SH,amihud_601288.SH,amihud_601328.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,20191231,-0.007242,0.002431,0.002614,0.000000,-0.006609,0.002024,-0.008065,-0.001773,-0.001698,...,0.000000,4.692472e-06,0.000002,0.000011,0.000005,0.000002,0.000013,0.000000,0.000020,0.000000
2019-12-30,20191230,-0.003608,0.001623,0.002621,0.006380,-0.000792,-0.010020,-0.002681,0.003559,-0.001695,...,0.000016,4.746218e-07,0.000004,0.000003,0.000007,0.000001,0.000020,0.000003,0.000011,0.000010
2019-12-27,20191227,0.009715,0.002441,0.005270,0.003200,-0.006299,0.000000,0.005391,0.003571,0.006826,...,0.000007,4.611380e-06,0.000000,0.000007,0.000007,0.000009,0.000006,0.000005,0.000008,0.000025
2019-12-26,20191226,0.010429,0.004085,0.005298,0.003210,0.008203,0.010121,0.010899,0.005386,0.005146,...,0.000012,1.122201e-05,0.000015,0.000023,0.000019,0.000010,0.000047,0.000024,0.000025,0.000113
2019-12-25,20191225,-0.006098,-0.003257,-0.005270,-0.004792,-0.005788,-0.007035,-0.002717,-0.001792,-0.005119,...,0.000023,6.526291e-06,0.000009,0.000008,0.000008,0.000011,0.000012,0.000009,0.000012,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-08,20190108,-0.008214,-0.002004,-0.005369,-0.001736,-0.011755,0.003955,-0.011204,0.003431,-0.007634,...,0.000007,1.738262e-05,0.000011,0.000016,0.000015,0.000013,0.000033,0.000000,0.000020,0.000024
2019-01-07,20190107,-0.001026,0.002008,-0.005340,-0.003460,0.000392,-0.001974,-0.005571,-0.005119,-0.005693,...,0.000009,3.371224e-07,0.000004,0.000007,0.000018,0.000011,0.000000,0.000008,0.000007,0.000042
2019-01-04,20190104,0.050647,0.015291,0.019048,0.019400,0.025322,0.023569,0.014124,0.013841,0.013462,...,0.000040,1.479300e-05,0.000033,0.000019,0.000031,0.000021,0.000069,0.000035,0.000019,0.000167


## 二、Var 向量值回归 amihud 和 yield rate

In [7]:
import numpy as np
import pandas as pd
import tushare as ts
from statsmodels.tsa import stattools
import matplotlib as mpl
import matplotlib.pyplot as plt


ts.set_token('094f15d71394516b730602faa77b1c708007b8d05df300590b4445ed')
pro = ts.pro_api()

# 计算自相关性
# stattools.acf(df.pct_chg, nlags=10)
# 计算偏自相关性
# stattools.pacf(df.pct_chg, nlags=10)



# df_1 = pro.daily(ts_code='000001.SZ', start_date='20190101')
# df_1.index = pd.to_datetime(df_1.trade_date)
# df_1 = df_1.sort_index(ascending=True)
# df_1

df = pd.read_csv('stock1.csv', index_col='trade_date')
df

,yield_rate_000001.SZ,yield_rate_600000.SH,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601166.SH,yield_rate_601288.SH,yield_rate_601328.SH,yield_rate_601398.SH,yield_rate_601818.SH,...,amihud_600016.SH,amihud_600036.SH,amihud_601166.SH,amihud_601288.SH,amihud_601328.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20191231,-0.007242,0.002431,0.002614,0.000000,-0.006609,0.002024,-0.008065,-0.001773,-0.001698,0.004556,...,0.000000,4.692472e-06,0.000002,0.000011,0.000005,0.000002,0.000013,0.000000,0.000020,0.000000
20191230,-0.003608,0.001623,0.002621,0.006380,-0.000792,-0.010020,-0.002681,0.003559,-0.001695,-0.011261,...,0.000016,4.746218e-07,0.000004,0.000003,0.000007,0.000001,0.000020,0.000003,0.000011,0.000010
20191227,0.009715,0.002441,0.005270,0.003200,-0.006299,0.000000,0.005391,0.003571,0.006826,0.002257,...,0.000007,4.611380e-06,0.000000,0.000007,0.000007,0.000009,0.000006,0.000005,0.000008,0.000025
20191226,0.010429,0.004085,0.005298,0.003210,0.008203,0.010121,0.010899,0.005386,0.005146,0.013730,...,0.000012,1.122201e-05,0.000015,0.000023,0.000019,0.000010,0.000047,0.000024,0.000025,0.000113
20191225,-0.006098,-0.003257,-0.005270,-0.004792,-0.005788,-0.007035,-0.002717,-0.001792,-0.005119,-0.002283,...,0.000023,6.526291e-06,0.000009,0.000008,0.000008,0.000011,0.000012,0.000009,0.000012,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190108,-0.008214,-0.002004,-0.005369,-0.001736,-0.011755,0.003955,-0.011204,0.003431,-0.007634,-0.005208,...,0.000007,1.738262e-05,0.000011,0.000016,0.000015,0.000013,0.000033,0.000000,0.000020,0.000024
20190107,-0.001026,0.002008,-0.005340,-0.003460,0.000392,-0.001974,-0.005571,-0.005119,-0.005693,0.000000,...,0.000009,3.371224e-07,0.000004,0.000007,0.000018,0.000011,0.000000,0.000008,0.000007,0.000042
20190104,0.050647,0.015291,0.019048,0.019400,0.025322,0.023569,0.014124,0.013841,0.013462,0.024000,...,0.000040,1.479300e-05,0.000033,0.000019,0.000031,0.000021,0.000069,0.000035,0.000019,0.000167


### 2.1 检验变量序列的平稳性

In [8]:
# ADF 检验时间序列的平稳性

import statsmodels.api as sm
import statsmodels.stats.diagnostic

# sm.tsa.stattools.adfuller(df['yield_rate_000001.SZ'])

def judge_percent(test):
    if test:
        if test[0]<=test[4]['1%']:
            return '1%'
        elif test[0]<=test[4]['5%']:
            return '5%'
        elif test[0]<=test[4]['10%']:
            return '10%'
        else:
            return None
    return None


column_list = [column for column in df]
column_test = {}
for column in column_list:
    test = sm.tsa.stattools.adfuller(df[column])
    temp_list = [test[1]]
    temp_list.append(judge_percent(test))
    column_test[column] = temp_list

column_test

{'yield_rate_000001.SZ': [5.298747073166892e-12, '1%'],
 'yield_rate_600000.SH': [2.665878317792005e-27, '1%'],
 'yield_rate_600015.SH': [5.915998749854795e-28, '1%'],
 'yield_rate_600016.SH': [2.370747073703703e-27, '1%'],
 'yield_rate_600036.SH': [1.9809980216697776e-29, '1%'],
 'yield_rate_601166.SH': [3.9638667260576014e-13, '1%'],
 'yield_rate_601288.SH': [1.747227738536033e-29, '1%'],
 'yield_rate_601328.SH': [1.0539370962238551e-28, '1%'],
 'yield_rate_601398.SH': [4.817418650125654e-29, '1%'],
 'yield_rate_601818.SH': [3.421479326385693e-28, '1%'],
 'yield_rate_601939.SH': [6.895166596879135e-28, '1%'],
 'yield_rate_601988.SH': [3.491500304993949e-29, '1%'],
 'yield_rate_601998.SH': [3.8234539499980044e-26, '1%'],
 'amihud_000001.SZ': [0.19233481254144114, None],
 'amihud_600000.SH': [0.1526575842526407, None],
 'amihud_600015.SH': [0.0012084137644636712, '1%'],
 'amihud_600016.SH': [9.029688889585918e-28, '1%'],
 'amihud_600036.SH': [0.0004484830669869233, '1%'],
 'amihud_6011

In [9]:
# 去除不满足 ADF 检验的对应时间序列

test_list_1=[]
test_list_2=[]
def test_column(column_test):
    for i in range(int(len(column_list)/2)):
        if column_test[column_list[i]][1]=='1%' and column_test[column_list[i+int(len(column_list)/2)]][1]=='1%':
            test_list_1.append(column_list[i])
            test_list_2.append(column_list[i+int(len(column_list)/2)])
    return test_list_1+test_list_2

df_1 = df[test_column(column_test)]
df_1

,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601288.SH,yield_rate_601398.SH,yield_rate_601818.SH,yield_rate_601939.SH,yield_rate_601988.SH,yield_rate_601998.SH,amihud_600015.SH,amihud_600016.SH,amihud_600036.SH,amihud_601288.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
trade_date,,,,,,,,,,,,,,,,,,
20191231,0.002614,0.000000,-0.006609,-0.008065,-0.001698,0.004556,0.000000,-0.005391,0.000000,0.000016,0.000000,4.692472e-06,0.000011,0.000002,0.000013,0.000000,0.000020,0.000000
20191230,0.002621,0.006380,-0.000792,-0.002681,-0.001695,-0.011261,-0.002759,0.005420,0.001623,0.000014,0.000016,4.746218e-07,0.000003,0.000001,0.000020,0.000003,0.000011,0.000010
20191227,0.005270,0.003200,-0.006299,0.005391,0.006826,0.002257,0.002766,0.002717,0.003257,0.000034,0.000007,4.611380e-06,0.000007,0.000009,0.000006,0.000005,0.000008,0.000025
20191226,0.005298,0.003210,0.008203,0.010899,0.005146,0.013730,0.008368,0.005464,0.006557,0.000062,0.000012,1.122201e-05,0.000023,0.000010,0.000047,0.000024,0.000025,0.000113
20191225,-0.005270,-0.004792,-0.005788,-0.002717,-0.005119,-0.002283,-0.002782,-0.002725,-0.004894,0.000086,0.000023,6.526291e-06,0.000008,0.000011,0.000012,0.000009,0.000012,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190108,-0.005369,-0.001736,-0.011755,-0.011204,-0.007634,-0.005208,0.000000,-0.008403,0.001845,0.000055,0.000007,1.738262e-05,0.000016,0.000013,0.000033,0.000000,0.000020,0.000024
20190107,-0.005340,-0.003460,0.000392,-0.005571,-0.005693,0.000000,-0.003145,-0.002793,-0.003676,0.000029,0.000009,3.371224e-07,0.000007,0.000011,0.000000,0.000008,0.000007,0.000042
20190104,0.019048,0.019400,0.025322,0.014124,0.013462,0.024000,0.015974,0.008451,0.018727,0.000118,0.000040,1.479300e-05,0.000019,0.000021,0.000069,0.000035,0.000019,0.000167


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from numpy import *
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller

import statsmodels.api as sm
import statsmodels.stats.diagnostic
from statsmodels.tsa.api import VAR


model = VAR(df_1)
results = model.fit(2)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:579: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [12]:
results.params

,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601288.SH,yield_rate_601398.SH,yield_rate_601818.SH,yield_rate_601939.SH,yield_rate_601988.SH,yield_rate_601998.SH,amihud_600015.SH,amihud_600016.SH,amihud_600036.SH,amihud_601288.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
const,0.004555,0.005092,0.005361,0.005010,0.005414,0.009108,0.006958,0.004777,0.006977,0.000033,0.000009,0.000005,0.000004,0.000004,0.000017,0.000009,0.000011,0.000035
L1.yield_rate_600015.SH,-0.066395,-0.140908,0.237393,-0.053430,-0.155427,-0.053557,-0.081797,0.052891,-0.331515,0.000398,0.000057,-0.000023,0.000094,0.000055,-0.000012,-0.000178,0.000131,0.000222
L1.yield_rate_600016.SH,0.411352,0.553205,0.365708,0.269543,0.304614,0.312859,0.214320,0.289501,0.861586,-0.000631,-0.000081,-0.000089,-0.000024,0.000070,-0.000028,0.000057,0.000022,-0.001024
L1.yield_rate_600036.SH,-0.103308,-0.168390,-0.225264,-0.097113,-0.137547,-0.163684,-0.082084,-0.093439,-0.159160,-0.000193,-0.000131,-0.000025,-0.000029,-0.000038,-0.000098,-0.000005,-0.000072,-0.000162
L1.yield_rate_601288.SH,-0.191584,-0.237114,-0.434876,-0.262751,-0.219828,-0.069291,-0.397073,-0.114619,-0.240663,-0.000213,-0.000193,-0.000006,-0.000039,-0.000058,0.000086,-0.000215,-0.000374,-0.000508
L1.yield_rate_601398.SH,0.183106,0.142831,-0.059125,0.126058,0.045333,-0.144880,0.240131,0.170926,0.090118,0.000157,0.000086,0.000140,-0.000051,-0.000050,-0.000187,-0.000144,0.000155,-0.001228
L1.yield_rate_601818.SH,0.047979,0.091797,0.215524,0.026260,0.071401,0.084871,0.067816,0.021373,0.163741,0.000395,0.000102,0.000054,0.000097,0.000055,0.000270,0.000182,0.000142,0.000765
L1.yield_rate_601939.SH,-0.077595,-0.000740,0.029433,0.039580,0.066967,0.199961,0.044615,-0.020909,0.039111,-0.000028,0.000145,-0.000009,-0.000039,0.000007,0.000174,0.000063,0.000010,0.000221
L1.yield_rate_601988.SH,-0.096179,-0.209158,-0.227976,-0.121268,0.049081,-0.262454,-0.109398,-0.316849,-0.326303,0.000315,0.000122,0.000054,0.000125,0.000058,0.000358,0.000366,0.000115,0.001412
L1.yield_rate_601998.SH,-0.087497,-0.083772,-0.050845,-0.031740,-0.098981,0.021215,-0.007259,-0.065734,-0.059721,-0.000145,-0.000042,-0.000076,-0.000103,-0.000076,-0.000233,-0.000100,-0.000147,0.000112


In [35]:
model.select_order()

In [31]:
results = model.fit(maxlags=11, ic='aic')

In [34]:
results.coefs

array([], shape=(0, 18, 18), dtype=float64)

In [39]:
results = model.fit(ic='aic')
results.params

,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601288.SH,yield_rate_601398.SH,yield_rate_601818.SH,yield_rate_601939.SH,yield_rate_601988.SH,yield_rate_601998.SH,amihud_600015.SH,amihud_600016.SH,amihud_600036.SH,amihud_601288.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
const,0.021572,0.020735,0.011573,0.011988,0.017363,0.015719,0.021023,0.018761,0.028205,-0.000025,0.000006,0.000001,0.000002,-5.372063e-07,-0.000011,1.126824e-07,0.000015,0.000020
L1.yield_rate_600015.SH,-0.116031,-0.312540,0.112547,0.006710,0.053130,-0.145960,-0.215184,0.184938,-0.202831,0.000824,0.000598,0.000015,0.000350,1.541083e-04,0.000022,-5.328154e-04,0.000917,0.001223
L1.yield_rate_600016.SH,1.166461,0.828891,1.365199,0.320707,0.183515,0.632962,0.440598,0.465691,0.811040,0.000059,-0.000205,-0.000257,-0.000378,1.130739e-04,0.000715,-3.489732e-04,-0.000015,0.000521
L1.yield_rate_600036.SH,0.061416,0.033258,0.028527,0.097297,-0.028688,0.056397,0.142035,0.064378,-0.043812,0.000392,0.000086,0.000045,0.000077,1.932646e-06,0.000286,2.527308e-04,0.000136,-0.000188
L1.yield_rate_601288.SH,-0.503276,-0.609874,-1.360204,-0.403581,-0.707650,-0.045463,-0.890759,-0.371308,-0.100015,-0.000433,-0.000207,-0.000229,0.000183,2.406511e-04,0.000362,3.135668e-04,0.000074,-0.000191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L11.amihud_601398.SH,-198.384422,20.053409,959.972144,-272.863755,-300.502540,-246.537812,-35.429928,-421.519464,-531.631087,1.288604,-0.145514,0.049472,-0.187075,2.330094e-01,0.502024,1.287536e-01,0.245850,1.776744
L11.amihud_601818.SH,44.888812,-27.246402,-67.495826,-28.552446,34.465875,51.059795,-125.990147,31.210082,138.398274,0.184292,0.013788,-0.045763,-0.009344,-1.819896e-02,-0.252639,-7.109078e-02,-0.101735,0.385480
L11.amihud_601939.SH,-32.733669,7.523387,86.540243,133.212688,289.794635,11.840157,114.387629,184.124550,43.333641,-0.749214,0.089254,-0.044803,0.060217,-2.464942e-01,-0.435159,-7.560366e-02,0.065388,-1.363135
L11.amihud_601988.SH,66.324048,-138.219822,5.323428,-128.288357,-56.958864,-151.154474,-87.219471,-96.339198,195.012387,0.590173,-0.226958,0.030333,-0.070972,-1.559698e-02,-0.265412,-1.271552e-01,-0.424818,-0.602859


### 2.2 模型的参数估计

### 2.3 确定最优的滞后阶数

### 2.4 Granger 因果检验

### 2.5 检验 Var 模型的平稳性

In [41]:
df = pd.read_csv('zong.csv', index_col = 'index')
model = VAR(df)
model.select_order(15)
results = model.fit(ic='aic')
results.params

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,CL,DC,DX,GY,GYSY,JR,KX,NY,RC,XX,YL
const,2.346166e-11,3.240048e-11,1.084064e-11,1.322079e-11,3.767874e-11,4.172635e-12,4.675862e-11,1.285054e-11,2.202713e-11,3.089461e-11,1.159737e-10
L1.CL,3.322336e-01,2.433892e-02,6.081287e-03,-1.356045e-02,1.212451e-01,6.468048e-03,1.096486e-01,1.644761e-02,3.829465e-03,7.326499e-03,1.730565e-02
L1.DC,4.008843e-02,4.577396e-01,-5.020242e-05,2.967150e-02,1.133171e-02,-1.948896e-03,6.891927e-02,-1.112514e-02,1.542080e-03,-8.321542e-02,6.044376e-02
L1.DX,7.275956e-02,7.015642e-03,3.939055e-01,2.282183e-02,-9.757522e-02,-2.690498e-02,6.788971e-02,-6.750478e-02,1.720306e-02,5.232805e-03,-2.169037e-01
L1.GY,2.765321e-02,9.092278e-02,-1.788739e-03,1.518826e-01,3.649918e-02,8.423375e-04,2.582113e-02,-1.746979e-02,9.550220e-04,3.560042e-02,-6.713636e-02
...,...,...,...,...,...,...,...,...,...,...,...
L17.KX,1.480704e-01,1.453473e-01,2.815749e-02,1.559614e-01,1.605924e-01,1.184508e-02,1.431932e-01,2.608941e-02,9.686937e-02,1.024082e-01,1.870894e-01
L17.NY,1.058701e-01,-3.324823e-02,2.708511e-02,7.793713e-02,2.786599e-01,1.188704e-02,2.996719e-01,3.385932e-02,1.218664e-01,-1.951154e-01,-8.663144e-02
L17.RC,-3.520738e-01,-2.902127e-01,-1.384791e-02,-3.449235e-01,-3.118529e-01,-6.861173e-03,-3.416018e-01,-3.790064e-02,-2.095342e-01,-8.455929e-02,-3.534270e-01
L17.XX,-2.997759e-03,1.520978e-02,-5.488651e-03,-1.049230e-02,2.287564e-02,3.078836e-03,6.004989e-02,-4.896441e-03,4.166821e-02,-6.684274e-03,-3.842005e-02


In [ ]:
# 商业银行
# 多元化银行
000001.SZ	平安银行
600000.SH	浦发银行
600015.SH	华夏银行
600016.SH	民生银行
600036.SH	招商银行
601166.SH	兴业银行
601288.SH	农业银行
601328.SH	交通银行
601398.SH	工商银行
# 601658.SH	邮储银行 时间过晚
601818.SH	光大银行
# 601916.SH	浙商银行 时间过晚
601939.SH	建设银行
601988.SH	中国银行
601998.SH	中信银行


# 区域银行
002142.SZ	宁波银行
002807.SZ	江阴银行
002839.SZ	张家港行
002936.SZ	郑州银行
002948.SZ	青岛银行
002958.SZ	青农商行
002966.SZ	苏州银行
600908.SH	无锡银行
600919.SH	江苏银行
600926.SH	杭州银行
600928.SH	西安银行
601009.SH	南京银行
601077.SH	渝农商行
601128.SH	常熟银行
601169.SH	北京银行
601229.SH	上海银行
601577.SH	长沙银行
601838.SH	成都银行
601860.SH	紫金银行
601997.SH	贵阳银行
603323.SH	苏农银行

In [66]:
a = np.arange(12).reshape(2,3,2)
b = np.arange(12,24).reshape(2,2,3)
print(a)
print(b)
c = np.matmul(a,b)
print(c)

[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]]
[[[12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]]]
[[[ 15  16  17]
  [ 69  74  79]
  [123 132 141]]

 [[255 268 281]
  [333 350 367]
  [411 432 453]]]


In [74]:
temp = np.identity(5)
temp = np.expand_dims(temp,0).repeat(2, axis=0)
temp

array([[[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]])

In [76]:
temp_1 = np.zeros((5,5))
for i in range(1,5):
    temp_1[i,i-1] = 1
temp_1

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [78]:
np.matmul(temp, temp_1)

array([[[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]]])

In [67]:
d = np.dot(a,b)
d

array([[[[ 15,  16,  17],
         [ 21,  22,  23]],

        [[ 69,  74,  79],
         [ 99, 104, 109]],

        [[123, 132, 141],
         [177, 186, 195]]],


       [[[177, 190, 203],
         [255, 268, 281]],

        [[231, 248, 265],
         [333, 350, 367]],

        [[285, 306, 327],
         [411, 432, 453]]]])

In [48]:
c.shape

(2, 3, 2, 3)

In [61]:
a=mat(np.reshape(np.arange(6),(2,3)))
b=np.reshape(np.arange(6),(3,2))
print(a)
print(b)
print(a*b)

[[0 1 2]
 [3 4 5]]
[[0 1]
 [2 3]
 [4 5]]
[[10 13]
 [28 40]]


In [63]:
print(type(a))
print(type(b))
print(type(a*b))

<class 'numpy.matrix'>
<class 'numpy.ndarray'>
<class 'numpy.matrix'>


In [53]:
a=np.reshape(a,(-1,3))

In [64]:
a

matrix([[0, 1, 2],
        [3, 4, 5]])

In [65]:
b

array([[0, 1],
       [2, 3],
       [4, 5]])

In [54]:
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17]])

In [55]:
a.reshape(2,3,3)

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]])